<a href="https://colab.research.google.com/github/puroong/2025-SSUML/blob/main/Term_Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
# print(IN_COLAB)

if IN_COLAB:
    !git clone https://github.com/ssuai/project_finance.git
    !git clone https://github.com/ssuai/ISLP.git
    #!pip install ISLP # This takes too much time

fatal: destination path 'project_finance' already exists and is not an empty directory.
Cloning into 'ISLP'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 63 (delta 14), reused 53 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (63/63), 2.34 MiB | 9.06 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [29]:
from ISLP.utils import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize)
import statsmodels.api as sm
from ISLP.models import sklearn_sm
from sklearn.model_selection import \
     (cross_validate,
      KFold,
      ShuffleSplit)

import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [5]:
import pandas as pd
from datetime import datetime

# helper function for reading datatset
def read_data(file_path):
    df = pd.read_csv(file_path)
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d') # convert it to datatime

    return df

df_train = read_data('project_finance/data/kospi_train.csv')
df_test = read_data('project_finance/data/kospi_test.csv')

len(df_train), len(df_test)

(986, 244)

In [ ]:
terms = df_train.columns.drop('Close').drop('Date').drop('Volume')

X = MS(terms).fit_transform(df_train)
y = df_train['Close']

model = sm.OLS(y, X)
result = model.fit()

summarize(result)

/content/ISLP/models/columns.py:151: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if is_categorical[i]:
/content/ISLP/models/columns.py:151: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if is_categorical[i]:
/content/ISLP/models/columns.py:151: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if is_categorical[i]:
/content/ISLP/models/columns.py:151: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a fut

,coef,std err,t,P>|t|
intercept,3.3326,1.825,1.826,0.068
Open,-0.6619,0.024,-27.198,0.000
Low,0.8633,0.019,46.195,0.000
High,0.7982,0.023,34.660,0.000


In [ ]:
hp_model = sklearn_sm(sm.OLS,
                      MS(['Open', 'Low', 'High']))

cv_results = cross_validate(hp_model,
                            X,
                            y,
                            cv=df_train.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

Streaming output truncated to the last 5000 lines.
/content/ISLP/models/columns.py:151: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if is_categorical[i]:
/content/ISLP/models/columns.py:151: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if is_categorical[i]:
/content/ISLP/models/columns.py:151: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if is_categorical[i]:
/content/ISLP/models/columns.py:151: FutureWarning: Series.__getitem__

np.float64(95.55412937893493)

In [77]:
import itertools

def column_combination():
  significant_columns = ['Open', 'Low', 'High', 'Close']

  for r in range(1, len(significant_columns)+1):
      l = itertools.combinations(significant_columns, r)
      for i in l:
          yield list(i)

result_dict = {}

X_test = None

for terms in [['Open', 'Low', 'High', 'Close']]:#column_combination():
#terms = df_train.columns.drop('Close').drop('Date').drop('Volume')
  ms = MS(terms)
  X_test = MS(terms).fit_transform(df_test)[:-1].reset_index(drop=True)
  y_test = df_test['Close'][1:].reset_index(drop=True)


  X = MS(terms).fit_transform(df_train)[:-1].reset_index(drop=True)
  y = df_train['Close'][1:].reset_index(drop=True)

  model = sm.OLS(y, X)
  result = model.fit()
  """
  hp_model = sklearn_sm(sm.OLS,
                      MS(terms))

  cv_results = cross_validate(hp_model,
                              X,
                              y,
                              cv=df_train.shape[0]-1)
  cv_err = np.mean(cv_results['test_score'])
  result_dict[str(terms)] = cv_err
  """

  y_pred = model.predict(X_test)
  #mse = mean_squared_error(y_test, y_pred)

  result_dict[str(terms)] = y_pred

y_pred.shape

/content/ISLP/models/columns.py:151: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if is_categorical[i]:
/content/ISLP/models/columns.py:151: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if is_categorical[i]:
/content/ISLP/models/columns.py:151: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if is_categorical[i]:
/content/ISLP/models/columns.py:151: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a fut

ValueError: shapes (985,5) and (243,5) not aligned: 5 (dim 1) != 243 (dim 0)

In [14]:
kospi_test = load_data("kospi_test")
terms = kospi_test.columns.drop('Close').drop('Date').drop('Volume')

new_X = MS(terms).fit_transform(kospi)[:-prev_days].reset_index(drop=True)

new_predictions = result.get_prediction(new_X);
new_predictions.predicted_mean

FileNotFoundError: [Errno 2] No such file or directory: '/content/ISLP/data/kospi_test.csv'

In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 1) 시계열 lagged feature 생성 함수
def make_lags(df, cols, prev_days=1):
    df_lag = df.copy()
    for c in cols:
        for lag in range(1, prev_days+1):
            df_lag[f"{c}_lag{lag}"] = df_lag[c].shift(lag)
    return df_lag.dropna()

# 2) 파라미터 설정
features  = ["Open", "High", "Low", "Close", "Volume"]
prev_days = 1

# 3) 학습/테스트 데이터에 lag 적용
train_lagged = make_lags(df_train, features, prev_days)
X_train      = train_lagged[[f"{c}_lag1" for c in features]]
y_train      = train_lagged["Close"]

test_lagged  = make_lags(df_test, features, prev_days)
X_test       = test_lagged[[f"{c}_lag1" for c in features]]
y_test       = test_lagged["Close"]

# 4) 후보 모델 정의
models = {
    "OLS":   LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.1),
    "SVR":   SVR(kernel="rbf", C=1.0, epsilon=0.1),
    "RF":    RandomForestRegressor(n_estimators=100, random_state=42)
}

# 5) 시계열 교차검증으로 모델 비교
tscv    = TimeSeriesSplit(n_splits=5)
results = {}
for name, model in models.items():
    neg_mse       = cross_val_score(model, X_train, y_train,
                                    cv=tscv,
                                    scoring="neg_mean_squared_error")
    results[name] = -neg_mse.mean()  # 양의 MSE 값

# CV 결과 출력
for name, mse in results.items():
    print(f"{name:6s} CV MSE: {mse:.4f}")

# 6) 최적 모델 선택 및 전체 학습
best_name  = min(results, key=results.get)
best_model = models[best_name]
best_model.fit(X_train, y_train)
print("선택된 모델:", best_name)

# 7) 테스트 세트 성능 평가
y_pred = best_model.predict(X_test)
mse    = mean_squared_error(y_test, y_pred)
mae    = mean_absolute_error(y_test, y_pred)
print(f"Test MSE: {mse:.4f}")
print(f"Test MAE: {mae:.4f}")

,Close
1,1993.699951
2,2010.250000
3,2037.099976
4,2025.270020
5,2064.709961
...,...
981,2313.689941
982,2317.139893
983,2332.790039
984,2280.449951
